# MGIMO intensive

## Text data base processing

Notebook is generated with help of DeepSeek with prompt:
```prompt
## Базовые параметры
- Роль: Junior Python Developer (Data Analyst)
- Специализация: data analysis, data visualization, NLP
- Уровень: начинающий
- Температура: 0 (максимальная точность и предсказуемость)

## Контекст выполнения
Разработка кода для предобработки и первичного анализа сырого текста по биоразнообразию

## Входные данные
- файл `text.txt` на диске
- в файле содержится отрывок из описания заповедника на русском языке

## Технические ограничения
- Использовать библиотеки `re`, `nltk`, `spacy` и стандартные библиотеки Python
- Код запускается в интерактивном ноутбуке Jupyter
- Визуализация и диаграммы для интерактивного ноутбука

## Требования к реализации

### Функциональные требования
1. Требуется провести анализ различными библиотеками и сравнить результаты
2. Требуется извлечение именованных сущностей и таблиц 
3. Визуализация wordcloud для описания текста 
4. Необходимо продемонстрировать несколько вариантов токенизации текста

### Технические требования
- Сложность кода: не используй классы, ограничься функциями
- Архитектура: упрощенная, для использования в интерактивных ноутбуках
- Стиль кода: PEP 8, black (длина строки 79)
- Документация: Docstrings в стиле Google
- Безопасность: Никаких захардкоженных credentials
```

### 1. Libraries install

In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_sm

### 2. Libraries and downloads

In [ ]:
import re
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize, word_tokenize
from wordcloud import WordCloud

# Загрузка необходимых ресурсов NLTK
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Загрузка русской модели spaCy
try:
    nlp = spacy.load('ru_core_news_sm')
except OSError:
    print("Модель 'ru_core_news_sm' не найдена.")
    print("Установите её командой: python -m spacy download ru_core_news_sm")
    # Можно использовать пустую модель для демонстрации, но NER не будет работать
    # nlp = spacy.blank('ru')

### 3. Utilities functions

In [ ]:
def load_text(filepath: str) -> str:
    """Загружает текст из файла.

    Args:
        filepath: Путь к текстовому файлу.

    Returns:
        Содержимое файла в виде строки.
    """
    path = Path(filepath)
    if not path.exists():
        raise FileNotFoundError(f"Файл {filepath} не найден.")
    return path.read_text(encoding='utf-8')


def clean_text(text: str) -> str:
    """Базовая очистка текста от лишних пробелов и спецсимволов.

    Args:
        text: Исходный текст.

    Returns:
        Очищенный текст.
    """
    # Замена множественных пробелов и переносов строк на один пробел
    text = re.sub(r'\s+', ' ', text)
    # Удаление символов, не являющихся буквами, цифрами или знаками препинания
    # Оставляем точки и запятые, так как они важны для токенизации предложений
    text = re.sub(r'[^а-яА-ЯёЁ0-9\s\.\,\-\–\(\)]', '', text)
    return text.strip()


def tokenize_with_nltk(text: str) -> tuple:
    """Токенизация текста с помощью NLTK.

    Args:
        text: Исходный текст.

    Returns:
        Кортеж (список предложений, список слов).
    """
    sentences = sent_tokenize(text, language='russian')
    words = word_tokenize(text.lower(), language='russian')
    # Оставляем только слова (без пунктуации)
    words = [word for word in words if word.isalpha()]
    return sentences, words


def tokenize_with_spacy(text: str) -> tuple:
    """Токенизация текста с помощью spaCy.

    Args:
        text: Исходный текст.

    Returns:
        Кортеж (список предложений, список слов).
    """
    if 'nlp' not in globals() or nlp is None:
        raise RuntimeError("Модель spaCy не загружена.")

    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    # Приводим к нижнему регистру и оставляем только слова
    words = [token.text.lower() for token in doc if token.is_alpha]
    return sentences, words


def compare_tokenization(text: str):
    """Сравнивает результаты токенизации NLTK и spaCy.

    Args:
        text: Исходный текст.
    """
    print("=" * 50)
    print("СРАВНЕНИЕ ТОКЕНИЗАЦИИ")
    print("=" * 50)

    # NLTK токенизация
    nltk_sentences, nltk_words = tokenize_with_nltk(text)
    print(f"\nNLTK:")
    print(f"  Предложений: {len(nltk_sentences)}")
    print(f"  Слов: {len(nltk_words)}")
    if nltk_sentences:
        print(f"  Первые 3 предложения: {nltk_sentences[:3]}")

    # spaCy токенизация
    try:
        spacy_sentences, spacy_words = tokenize_with_spacy(text)
        print(f"\nspaCy:")
        print(f"  Предложений: {len(spacy_sentences)}")
        print(f"  Слов: {len(spacy_words)}")
        if spacy_sentences:
            print(f"  Первые 3 предложения: {spacy_sentences[:3]}")
    except RuntimeError as e:
        print(f"\nspaCy: {e}")


def extract_entities_spacy(text: str) -> list:
    """Извлекает именованные сущности с помощью spaCy.

    Args:
        text: Исходный текст.

    Returns:
        Список кортежей (текст_сущности, тип_сущности).
    """
    if 'nlp' not in globals() or nlp is None:
        raise RuntimeError("Модель spaCy не загружена. NER недоступен.")

    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities


def extract_entities_regex(text: str) -> dict:
    """Извлекает потенциальные названия видов с помощью регулярных выражений.

    Args:
        text: Исходный текст.

    Returns:
        Словарь с типами сущностей и найденными значениями.
    """
    # Простой поиск слов с заглавной буквы (потенциальные названия видов)
    # Ищем слова длиной больше 2 символов, начинающиеся с заглавной буквы
    potential_species = re.findall(r'\b[А-ЯЁ][а-яё]{2,}(?:\s+[А-ЯЁ][а-яё]{2,})*\b', text)

    # Поиск числовых значений (площади, количества)
    numbers = re.findall(r'\b\d+(?:[.,]\d+)?\s*(?:км|га|м|тыс|млн)\b', text)

    return {
        'potential_species': potential_species,
        'measurements': numbers
    }


def create_wordcloud(words: list, title: str = "WordCloud") -> None:
    """Создает и отображает облако слов.

    Args:
        words: Список слов для облака.
        title: Заголовок графика.
    """
    if not words:
        print("Недостаточно слов для создания wordcloud.")
        return

    # Преобразуем список слов в строку
    text_for_cloud = ' '.join(words)

    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color='white',
        colormap='viridis',
        max_words=100,
        contour_width=1,
        contour_color='steelblue'
    ).generate(text_for_cloud)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=16)
    plt.show()


def get_stopwords() -> set:
    """Получает список стоп-слов для русского языка.

    Returns:
        Множество стоп-слов.
    """
    russian_stopwords = set(stopwords.words('russian'))
    # Добавляем дополнительные часто встречающиеся слова
    extra_stops = {'это', 'весь', 'также', 'например', 'который', 'свой',
                   'очень', 'ещё', 'уже', 'чтобы', 'может', 'быть'}
    return russian_stopwords.union(extra_stops)


def plot_frequency_distribution(words: list, top_n: int = 20) -> None:
    """Строит график частотности слов.

    Args:
        words: Список слов.
        top_n: Количество наиболее частотных слов для отображения.
    """
    stop_words = get_stopwords()
    # Фильтруем стоп-слова и короткие слова
    filtered_words = [word for word in words if word not in stop_words and len(word) > 2]

    freq_dist = FreqDist(filtered_words)
    common_words = freq_dist.most_common(top_n)

    if not common_words:
        print("Нет данных для построения графика частотности.")
        return

    words_list, counts = zip(*common_words)

    plt.figure(figsize=(12, 6))
    plt.bar(range(len(words_list)), counts, tick_label=words_list)
    plt.xticks(rotation=45, ha='right')
    plt.xlabel('Слова')
    plt.ylabel('Частота')
    plt.title(f'Топ-{top_n} наиболее частотных слов')
    plt.tight_layout()
    plt.show()

### 4. Text processing examples

In [ ]:
# Загрузка текста
text = load_text('text.txt')

print("ИСХОДНЫЙ ТЕКСТ (первые 500 символов):")
print("-" * 50)
print(text[:500])
print("-" * 50)

# Очистка текста
cleaned_text = clean_text(text)
print(f"\nДлина исходного текста: {len(text)} символов")
print(f"Длина после очистки: {len(cleaned_text)} символов")

# 1. Сравнение токенизации
compare_tokenization(cleaned_text)

In [ ]:
# 2. Извлечение именованных сущностей
print("\n" + "=" * 50)
print("ИЗВЛЕЧЕНИЕ СУЩНОСТЕЙ")
print("=" * 50)

# Извлечение с помощью регулярных выражений
regex_entities = extract_entities_regex(cleaned_text)
print("\nРегулярные выражения:")
for entity_type, values in regex_entities.items():
    if values:
        print(f"  {entity_type}: {values[:10]}")  # Показываем первые 10

# Извлечение с помощью spaCy NER
try:
    spacy_entities = extract_entities_spacy(cleaned_text)
    print("\nspaCy NER:")
    if spacy_entities:
        # Группируем по типам для наглядности
        entity_types = {}
        for entity, label in spacy_entities:
            if label not in entity_types:
                entity_types[label] = []
            entity_types[label].append(entity)

        for label, entities in entity_types.items():
            print(f"  {label}: {entities[:5]}")  # Показываем первые 5 каждого типа
    else:
        print("  Сущности не найдены.")
except RuntimeError as e:
    print(f"\nspaCy NER: {e}")

In [ ]:
# 3. Токенизация для визуализации
# Используем NLTK для визуализации, так как он всегда доступен
sentences, words = tokenize_with_nltk(cleaned_text)

# 4. WordCloud
print("\n" + "=" * 50)
print("ВИЗУАЛИЗАЦИЯ")
print("=" * 50)

# Создаем облако слов (без стоп-слов)
stop_words = get_stopwords()
filtered_words = [word for word in words if word not in stop_words and len(word) > 2]
create_wordcloud(filtered_words, title="Облако ключевых слов текста")

# График частотности
plot_frequency_distribution(words)

# Дополнительная информация
print("\n" + "=" * 50)
print("СТАТИСТИКА")
print("=" * 50)
print(f"Всего слов (после токенизации): {len(words)}")
print(f"Уникальных слов: {len(set(words))}")
print(f"Всего предложений: {len(sentences)}")
print(f"Средняя длина предложения: {len(words) / len(sentences):.1f} слов")

In [ ]:
print(cleaned_text[:1000])

In [ ]:
print(stop_words)

In [ ]:
print(words[:100])